In [1]:
import sys
sys.path.append("../src")
import numpy as np
import nltk
from nltk.tag import pos_tag, map_tag
from nltk.tree import Tree
import matplotlib.pyplot as plt
from collections import defaultdict
import pandas as pd
from iparse_vq import IParser
import evaluate
import torch
import torch_struct
import re

In [2]:
# iparse = IParser("../models/en_label_gpt2_medium_cat256")

model = '32'

iparse = IParser(f"../models/{model}.pt")

In [3]:
# cats as both POS and Word
#     also simple pos

def rewrite_tree(treebank_tree, cats, replace_both=False):
    
    # Treebank tree must be an NLTK tree
    
    tree = treebank_tree.copy(deep=True)
    assert len(tree.leaves()) == len(cats), 'Size of tree is different from number of categories'
    for i, leafPos in enumerate(tree.treepositions('leaves')):
        if replace_both:
            tree[leafPos[:-1]] = Tree(str(cats[i]), [str(cats[i])])
        else:
            tree[leafPos] = str(cats[i])

# #         using cats as POS, keeping word
#         tree[leafPos[:-1]] = Tree(str(cats[i]), [tree[leafPos]])
    return tree

def output_string(treebank_tree):
    return re.sub(r'\s+', ' ', str(treebank_tree).replace('\n', '').strip()) + '\n'

In [6]:
dev_treebank = iparse.load_dev(max_len_dev=999999)

dev_predicted_and_cats, encoded = iparse.parser.parse(
    dev_treebank.without_gold_annotations(),
    subbatch_max_tokens=500,
    tau=0.0,
    return_cats=True,
    return_encoded=True
)

/Users/thomas/anaconda3/lib/python3.8/site-packages/torch/distributions/distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


In [7]:
# # dev_treebank = iparse.load_dev(max_len_dev=999999)

# with open(f'simple_pos_test_only.txt', 'a') as the_file:
#     for dev_tree in dev_treebank:
#         cats = [map_tag('en-ptb', 'universal', pos[1]) for pos in dev_tree.tree.pos()]
#         induced_tree = rewrite_tree(dev_tree.tree, cats, replace_both=True)
#         the_file.write(output_string(induced_tree))

In [8]:
with open(f'dcat_gold_test_{model}_only.txt', 'w') as the_file:
    for dev_tree, (tree, cats) in zip(dev_treebank, dev_predicted_and_cats):
        induced_tree = rewrite_tree(dev_tree.tree, cats, replace_both=True)
        the_file.write(output_string(induced_tree))
        


In [9]:
with open(f'dcat_test_{model}_only.txt', 'w') as the_file:
    for (tree, cats) in dev_predicted_and_cats:
        induced_tree = rewrite_tree(tree, cats, replace_both=True)
        the_file.write(output_string(induced_tree))

In [10]:
del dev_treebank
del dev_predicted_and_cats
del encoded

In [4]:
train_treebank = iparse.load_train(max_len_dev=40)

train_predicted_and_cats, encoded = iparse.parser.parse(
    train_treebank.without_gold_annotations(),
    subbatch_max_tokens=500,
    tau=0.0,
    return_cats=True,
    return_encoded=True
)

/Users/thomas/anaconda3/lib/python3.8/site-packages/torch/distributions/distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


In [5]:
with open(f'dcat_train_{model}_only.txt', 'w') as the_file:
    for (tree, cats) in train_predicted_and_cats:
        induced_tree = rewrite_tree(tree, cats, replace_both=True)
        the_file.write(output_string(induced_tree))

In [11]:
# with open(f'simple_pos_train_only.txt', 'a') as the_file:
#     for train_tree in train_treebank:
#         cats = [map_tag('en-ptb', 'universal', pos[1]) for pos in train_tree.tree.pos()]
#         induced_tree = rewrite_tree(train_tree.tree, cats, replace_both=True)
#         the_file.write(output_string(induced_tree))

In [12]:
dev_treebank = iparse.load_dev(max_len_dev=999999)

with open(f'pos_test_only.txt', 'a') as the_file:
    for dev_tree in dev_treebank:
        cats = [pos[1] for pos in dev_tree.tree.pos()]
        induced_tree = rewrite_tree(dev_tree.tree, cats, replace_both=True)
        the_file.write(output_string(induced_tree))
        
train_treebank = iparse.load_train(max_len_dev=40)

with open(f'pos_train_only.txt', 'a') as the_file:
    for train_tree in train_treebank:
        cats = [pos[1] for pos in train_tree.tree.pos()]
        induced_tree = rewrite_tree(train_tree.tree, cats, replace_both=True)
        the_file.write(output_string(induced_tree))

In [ ]:
david hall thing